# Silver 계층 테이블 - 로봇 이벤트
![](img/silver_diagram.png)

이 스크립트는 Databricks Delta Live Tables에서 **세 개의 Silver 계층 테이블**을 정의하며, Bronze 계층(`bronze_robot_events`) 위에 구축됩니다.  
**데이터 품질 검사**, **정제**, **스키마 평탄화**를 적용하여 다운스트림 분석을 위한 데이터를 제공합니다.

---

## 1. `silver_command_events`
**목적:** 정제되고 필터링된 로봇 명령 실행 이벤트  
- **품질 규칙**:
  - 유효하지 않은 `duration_ms`(0보다 큰 값이 아님) 행 제거
  - `battery_level`이 0~100 사이인지 확인
- **변환**:
  - `event_type = command_execution`으로 필터링
  - 중첩 필드(명령 세부정보, 위치) 평탄화
  - 유효한 타임스탬프 보장
- **출력 컬럼**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

## 2. `silver_sensor_data`
**목적:** 정제된 로봇 센서 데이터  
- **품질 규칙**:
  - `battery_level`이 0~100 사이인지 확인
  - `battery_temp_c`가 0~50 사이인지 확인
- **변환**:
  - 배터리 및 LiDAR 센서 데이터 추출
  - 유효한 타임스탬프 보장
- **출력 컬럼**:
  - `robot_id`, `timestamp`, `battery_level`, `battery_temp_c`, `obstacle_detected`, `scan_points`

---

## 3. `silver_combined_events`
**목적:** 명령, 센서, 위치 데이터를 결합한 통합 테이블  
- **품질 규칙**:
  - 유효하지 않은 `duration_ms`(0보다 큰 값이 아님) 행 제거
  - `battery_level`이 0~100 사이인지 확인
- **변환**:
  - `event_type = command_execution`으로 필터링
  - 명령 + 센서 + 위치 필드 평탄화 및 병합
  - 유효한 타임스탬프 보장
- **출력 컬럼**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

> ⚡ **팁:** 실행 전 환경에 맞게 `CATALOG`, `BRONZE`, `SILVER` 변수를 업데이트하세요.

In [0]:
import dlt
from pyspark.sql.functions import col, when, to_timestamp

CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_command_events", comment="Cleaned and filtered command events")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_in_range", "battery_level BETWEEN 0 AND 100")
def silver_command_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    # Filter
    df = df.filter(col("event.event_type") == "command_execution")

    # Cleansing and flatten
    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))

    # Timestamp filtering
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "location.x",
        "location.y",
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_sensor_data", comment="Cleaned robot sensor data")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_sensor_data():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.withColumn("battery_temp_c", when((col("sensors.battery.temperature_c") >= 0) & (col("sensors.battery.temperature_c") <= 50), col("sensors.battery.temperature_c")).otherwise(None))

    df = df.filter(col("sensors.lidar.obstacle_detected").isNotNull())
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "battery_level",
        "battery_temp_c",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected"),
        col("sensors.lidar.scan_points").alias("scan_points")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_combined_events", comment="Combined robot command + sensor + location data")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_combined_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.filter(col("event.event_type") == "command_execution")

    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        col("location.x").alias("x"),
        col("location.y").alias("y"),
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )